In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Aniridia/GSE137995'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Conjunctival mRNA and miRNA expression profiles in congenital aniridia are genotype and phenotype dependent (AKK miRNA)"
!Series_summary	"Purpose:"
!Series_summary	"To evaluate conjunctival cell microRNA and mRNA expression in relation to observed phenotype and genotype of aniridia-associated keratopathy (AAK) in a cohort of subjects with congenital aniridia."
!Series_summary	"Methods:"
!Series_summary	"Using impression cytology, bulbar conjunctival cells were sampled from 20 subjects with congenital aniridia and 20 age and sex-matched healthy control subjects. RNA was extracted and microRNA and mRNA analysis was performed using microarrays. Results were related to the presence and severity of AAK determined by a standardized clinical grading scale and to the genotype (PAX6 mutation?) determined by clinical genetics."
!Series_summary	"Results:"
!Series_summary	"Of the 2549 microRNAs analyzed, 21 were differentially expressed relative to controls. 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Determine if the dataset contains gene expression data
is_gene_available = True

# Determine the availability and row keys for each variable
trait_row = 2  # Based on the 'disease: AAK', 'disease: healthy control'
age_row = 0
gender_row = 1

# Define the conversion functions
def convert_trait(value):
    value = value.split(': ')[1]
    if value == "AAK":
        return 1
    elif value == "healthy control":
        return 0
    else:
        return None

def convert_age(value):
    try:
        return int(value.split(': ')[1])
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(': ')[1]
    if value == "F":
        return 0
    elif value == "M":
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE137995', './preprocessed/Aniridia/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available, process and save it
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Aniridia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Aniridia/trait_data/GSE137995.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4096349': [1, 20, 0], 'GSM4096350': [1, 20, 0], 'GSM4096351': [1, 28, 0], 'GSM4096352': [1, 20, 0], 'GSM4096353': [1, 38, 0], 'GSM4096354': [1, 57, 1], 'GSM4096355': [1, 26, 0], 'GSM4096356': [1, 18, 1], 'GSM4096357': [1, 36, 0], 'GSM4096358': [1, 42, 0], 'GSM4096359': [1, 18, 0], 'GSM4096360': [1, 42, 0], 'GSM4096361': [1, 36, 1], 'GSM4096362': [1, 28, 0], 'GSM4096363': [1, 55, 0], 'GSM4096364': [1, 54, 1], 'GSM4096365': [1, 34, 1], 'GSM4096366': [1, 51, 0], 'GSM4096367': [1, 46, 0], 'GSM4096368': [1, 52, 0], 'GSM4096369': [0, 53, None], 'GSM4096370': [0, 54, 1], 'GSM4096371': [0, 40, None], 'GSM4096372': [0, 55, None], 'GSM4096373': [0, 57, None], 'GSM4096374': [0, 28, None], 'GSM4096375': [0, 39, None], 'GSM4096376': [0, 59, None], 'GSM4096377': [0, 20, None], 'GSM4096378': [0, 32, 1], 'GSM4096379': [0, 37, 1], 'GSM4096380': [0, 34, None], 'GSM4096381': [0, 28, None], 'GSM4096382': [0, 28, None], 'GSM4096383': [0, 29, 1], 'GSM4096384': [0, 19, None], 'GSM4096385': [0, 25, None]

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['hsa-let-7a-3p', 'hsa-let-7a-5p', 'hsa-let-7b-3p', 'hsa-let-7b-5p',
       'hsa-let-7c-3p', 'hsa-let-7c-5p', 'hsa-let-7d-3p', 'hsa-let-7d-5p',
       'hsa-let-7e-3p', 'hsa-let-7e-5p', 'hsa-let-7f-1-3p', 'hsa-let-7f-2-3p',
       'hsa-let-7f-5p', 'hsa-let-7g-3p', 'hsa-let-7g-5p', 'hsa-let-7i-3p',
       'hsa-let-7i-5p', 'hsa-miR-1-3p', 'hsa-miR-1-5p', 'hsa-miR-100-3p'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# requires_gene_mapping is False, so skipping STEP6


Gene annotation preview:
{'ID': ['hsa-miR-3132', 'hsa-miR-449c-5p', 'hsa-miR-4481', 'hsa-miR-6788-5p', 'hsa-miR-514a-3p'], 'miRNA_ID': ['hsa-miR-3132', 'hsa-miR-449c-5p', 'hsa-miR-4481', 'hsa-miR-6788-5p', 'hsa-miR-514a-3p'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Aniridia/gene_data/GSE137995.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Aniridia')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE137995', './preprocessed/Aniridia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Aniridia/GSE137995.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
